In [1]:
import cv2
import random
import numpy as np
import pywt
import matplotlib.pyplot as plt

In [2]:
def create_overlapping_blocks (image, height_o, width_o) :
    
    overlapping_blocks = np.zeros((height_o * width_o, 8, 8) , np.uint8)
    block_index = 0
    
    for y in range (height_o):
        for x in range (width_o):
      
            block = image [y : y+8, x : x+8]
            overlapping_blocks[block_index] = block
            block_index += 1
    
    return overlapping_blocks

In [3]:
def get_DWT_LL(overlapping_blocks, height_o, width_o):
    
    LL = np.zeros(( height_o * width_o, 4, 4 ), dtype = np.float_)
    
    for i in range (height_o * width_o):
        
        LL[i] ,(LH, HL, HH)= pywt.dwt2(overlapping_blocks[i], "haar")
    
    return LL


In [4]:
def get_singular(LL, height_o, width_o):
    
    singular = np.zeros((height_o * width_o), dtype = np.float_)
    
    for i in range (height_o * width_o):
        U, S, VT = np.linalg.svd(LL[i])
        singular[i] = S[0]
    
    return singular

In [5]:
def create_key(height_o, width_o):
    
    key = np.zeros((height_o, width_o, 2), np.uint32)
    key = np.random.randint(0, height_o * width_o, size = key.shape)
    return key

In [6]:
def create_master_share(key, singular, height_o, width_o):
    
    master_share = np.zeros((height_o, width_o), dtype = np.uint8)
    
    for y in range (height_o) :
        for x in range (width_o) :

            key_1, key_2 = key[y][x][0], key[y][x][1]
            if (singular[key_1] - singular[key_2]) > 0 :
                master_share[y][x] = 0 
            else :
                master_share[y][x] = 1
    
    return master_share

In [7]:
def create_binary_watermark(watermark, height_watermark, width_watermark):
    
    for y in range(height_watermark):
        for x in range (width_watermark):
            if watermark[y][x] > 127:
                watermark[y][x] = 1
            else :
                watermark[y][x] = 0
    return watermark

In [8]:
def xor_augmented_watermark(watermark, height_o, width_o, height_watermark, width_watermark):
    
    augmented_watermark = np.zeros((height_o, width_o), dtype = np.uint8)
    augmented_watermark[: height_watermark , : width_watermark] = watermark

    return augmented_watermark

In [9]:
def create_ownership(augmented_watermark, master_share):
    
    ownership = cv2.bitwise_xor(augmented_watermark, master_share)
    return ownership
    

In [10]:
def undo_watermark(ownership, master_share, height_watermark, width_watermark):
    
    undo_watermark = cv2.bitwise_xor(ownership, master_share)
    reduce_undo_watermark = np.zeros((height_watermark, width_watermark), dtype = np.uint8)
    reduce_undo_watermark = undo_watermark [: height_watermark , : width_watermark]

    for y in range(reduce_undo_watermark.shape[0]):
        for x in range(reduce_undo_watermark.shape[1]):
            if reduce_undo_watermark[y][x] == 1:
                reduce_undo_watermark[y][x] = 255
    
    return reduce_undo_watermark


In [11]:
def add_gaussian_noise(image, mean = 0, std_dev = 25):
   
    noise = np.random.normal(mean, std_dev, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise, dtype = cv2.CV_8U)
    
    return noisy_image


In [12]:
def motion_image(image, height, width):
    
    dx, dy = 5, 5
    
    transformation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])

    motion_image = cv2.warpAffine(image, transformation_matrix, (image.shape[1], image.shape[0]))

    return motion_image

In [13]:
def resize_image(image, height, width):

    resized_image = cv2.resize(image, ( width // 2, height // 2))
    return resized_image
   

In [14]:
def bmp_to_jpeg(image_bmp):
   
    image_jpeg  = 'image_JPEG.jpg'

    cv2.imwrite(image_jpeg, image_bmp, [int(cv2.IMWRITE_JPEG_QUALITY), 10])
    image = cv2.imread(image_jpeg, 0)
    return  image

In [15]:
def rotated_image(image, height, width):
    
  
    angle = 1

    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)

    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    return rotated_image

In [16]:
image = cv2.imread("lena.bmp", 0)
height, width = image.shape
watermark = cv2.imread("letterA.bmp",0)
height_watermark, width_watermark = watermark.shape
watermark = create_binary_watermark(watermark, height_watermark, width_watermark)
height_o, width_o = height - 7, width - 7 # overlapping height and width
key = create_key(height_o, width_o)


In [17]:

overlapping_blocks = create_overlapping_blocks(image, height_o, width_o)
LL = get_DWT_LL(overlapping_blocks, height_o, width_o)
singular = get_singular(LL, height_o, width_o)
master_share = create_master_share(key,singular, height_o, width_o)
augmented_watermark = xor_augmented_watermark(watermark, height_o, width_o, height_watermark, width_watermark)
ownership = create_ownership(augmented_watermark, master_share)

print(ownership)
cv2.imwrite("ownership.bmp",ownership)

[[0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 [0 1 0 ... 1 0 1]
 ...
 [0 0 1 ... 1 0 1]
 [0 0 0 ... 1 1 1]
 [1 1 0 ... 1 0 0]]


True

In [18]:
reduce_undo_watermark = undo_watermark(ownership, master_share, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark.bmp",reduce_undo_watermark)
print(reduce_undo_watermark)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [19]:

reduce_undo_binary_watermark =  create_binary_watermark(reduce_undo_watermark, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo watermark ber : " ,ber)


original watermark and undo watermark ber :  0.0


In [20]:
image_noisy = add_gaussian_noise(image)
cv2.imwrite("image_noisy.bmp", image_noisy)

height_noisy, width_noisy = image_noisy.shape
height_noisy_o, width_noisy_o = height_noisy - 7, width_noisy - 7 # overlapping height and width

overlapping_blocks_noisy = create_overlapping_blocks(image_noisy, height_noisy_o, width_noisy_o)
LL_noisy = get_DWT_LL(overlapping_blocks_noisy, height_noisy_o, width_noisy_o)
singular_noisy = get_singular(LL_noisy, height_noisy_o, width_noisy_o)

master_share_noisy = create_master_share(key, singular_noisy, height_noisy_o, width_noisy_o)
reduce_undo_watermark_noisy = undo_watermark(ownership, master_share_noisy, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark_noisy.bmp",reduce_undo_watermark_noisy)
print(reduce_undo_watermark_noisy)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255   0 ... 255   0 255]
 ...
 [255 255 255 ... 255 255 255]
 [  0 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [21]:
reduce_undo_binary_watermark_noisy =  create_binary_watermark(reduce_undo_watermark_noisy, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark_noisy)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo noisy watermark ber : " ,ber)

original watermark and undo noisy watermark ber :  0.09552001953125


In [22]:
image_motion = motion_image(image, height, width)
cv2.imwrite("image_motion.bmp", image_motion)


height_motion, width_motion = image_motion.shape
height_motion_o, width_motion_o = height_motion - 7, width_motion - 7 # overlapping height and width

overlapping_blocks_motion = create_overlapping_blocks(image_motion, height_motion_o, width_motion_o)
LL_motion = get_DWT_LL(overlapping_blocks_motion, height_motion_o, width_motion_o)
singular_motion = get_singular(LL_motion, height_motion_o, width_motion_o)

master_share_motion = create_master_share(key, singular_motion, height_motion_o, width_motion_o)
reduce_undo_watermark_motion = undo_watermark(ownership, master_share_motion, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark_motion.bmp",reduce_undo_watermark_motion)
print(reduce_undo_watermark_motion)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255   0 ... 255 255 255]
 ...
 [255   0 255 ... 255 255 255]
 [  0 255 255 ... 255 255 255]
 [255   0 255 ... 255 255 255]]


In [23]:
reduce_undo_binary_watermark_motion =  create_binary_watermark(reduce_undo_watermark_motion, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark_motion)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo motion watermark ber : " ,ber)

original watermark and undo motion watermark ber :  0.14373779296875


In [24]:
image_resized_small = resize_image(image, height, width)
cv2.imwrite("image_resized.bmp", image_resized_small)
image_resized = cv2.resize(image_resized_small, (width, height), interpolation=cv2.INTER_CUBIC)

height_resized, width_resized = image_resized.shape
height_resized_o, width_resized_o = height_resized - 7, width_resized - 7 # overlapping height and width

overlapping_blocks_resized = create_overlapping_blocks(image_resized, height_resized_o, width_resized_o)
LL_resized = get_DWT_LL(overlapping_blocks_resized, height_resized_o, width_resized_o)
singular_resized = get_singular(LL_resized, height_resized_o, width_resized_o)

master_share_resized = create_master_share(key, singular_resized, height_resized_o, width_resized_o)
reduce_undo_watermark_resized = undo_watermark(ownership, master_share_resized, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark_resized.bmp",reduce_undo_watermark_resized)
print(reduce_undo_watermark_resized)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [25]:
reduce_undo_binary_watermark_resized =  create_binary_watermark(reduce_undo_watermark_resized, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark_resized)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo resized watermark ber : " ,ber)

original watermark and undo resized watermark ber :  0.00250244140625


In [26]:
image_jpeg = bmp_to_jpeg(image)
cv2.imwrite("image_jpeg.jpeg", image_jpeg)


height_jpeg, width_jpeg = image_jpeg.shape
height_jpeg_o, width_jpeg_o = height_jpeg - 7, width_jpeg - 7 # overlapping height and width

overlapping_blocks_jpeg = create_overlapping_blocks(image_jpeg, height_jpeg_o, width_jpeg_o)
LL_jpeg = get_DWT_LL(overlapping_blocks_jpeg, height_jpeg_o, width_jpeg_o)
singular_jpeg = get_singular(LL_jpeg, height_jpeg_o, width_jpeg_o)

master_share_jpeg = create_master_share(key, singular_jpeg, height_jpeg_o, width_jpeg_o)
reduce_undo_watermark_jpeg= undo_watermark(ownership, master_share_jpeg, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark_jpeg.bmp",reduce_undo_watermark_jpeg)
print(reduce_undo_watermark_jpeg)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [27]:
reduce_undo_binary_watermark_jpeg =  create_binary_watermark(reduce_undo_watermark_jpeg, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark_jpeg)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo jpeg watermark ber : " ,ber)

original watermark and undo jpeg watermark ber :  0.01580810546875


In [28]:
image_rotated = rotated_image (image, height, width)
cv2.imwrite("Image_rotated.bmp", image_rotated)

height_rotated, width_rotated = image_rotated.shape
height_rotated_o, width_rotated_o = height_rotated - 7, width_rotated - 7 # overlapping height and width

overlapping_blocks_rotated = create_overlapping_blocks(image_rotated, height_rotated_o, width_rotated_o)
LL_rotated = get_DWT_LL(overlapping_blocks_rotated, height_rotated_o, width_rotated_o)
singular_rotated = get_singular(LL_rotated, height_rotated_o, width_rotated_o)

master_share_rotated = create_master_share(key, singular_rotated, height_rotated_o, width_rotated_o)
reduce_undo_watermark_rotated= undo_watermark(ownership, master_share_rotated, height_watermark, width_watermark)
cv2.imwrite("reduce_undo_watermark_rotated.bmp",reduce_undo_watermark_rotated)
print(reduce_undo_watermark_rotated)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255   0 255]
 [  0 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255   0 255 ... 255 255 255]]


In [29]:
reduce_undo_binary_watermark_rotated = create_binary_watermark(reduce_undo_watermark_rotated, height_watermark, width_watermark)
diff = cv2.absdiff(watermark, reduce_undo_binary_watermark_rotated)

num_errors = cv2.countNonZero(diff)
total_pixels = height_watermark * width_watermark # Total number of pixels
ber = num_errors / total_pixels
print("original watermark and undo rotated watermark ber : " ,ber)

original watermark and undo rotated watermark ber :  0.0589599609375
